## Configure where to fetch our training data

In [1]:
 # An S3 Bucket Name
data_bucket_name='s3webcamuploader83a65c76f8384092b63d212639122190'

# A prefix name inside the S3 bucket containing sub-folders of images (one per label class)
dataset_name = 'websummit-live-demo' 

## Setting up the environment

In [4]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

role = get_execution_role()
sess = sagemaker.Session()

training_image = get_image_uri(sess.boto_region_name, 'image-classification', repo_version="latest")

## Preparing data for our model

In [5]:
# Find im2rec in our environment and set up some other vars in our environemnt

base_dir='/tmp'

%env BASE_DIR=$base_dir
%env S3_DATA_BUCKET_NAME = $data_bucket_name
%env DATASET_NAME = $dataset_name


import sys,os

suffix='/mxnet/tools/im2rec.py'
im2rec = list(filter( (lambda x: os.path.isfile(x + suffix )), sys.path))[0] + suffix
%env IM2REC=$im2rec

env: BASE_DIR=/tmp
env: S3_DATA_BUCKET_NAME=s3webcamuploader83a65c76f8384092b63d212639122190
env: DATASET_NAME=websummit-live-demo
env: IM2REC=/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/mxnet/tools/im2rec.py


In [6]:
%%bash
# Pull our images from S3
set -x
aws s3 sync s3://$S3_DATA_BUCKET_NAME/public/$DATASET_NAME $BASE_DIR/$DATASET_NAME --quiet

+ aws s3 sync s3://s3webcamuploader83a65c76f8384092b63d212639122190/public/websummit-live-demo /tmp/websummit-live-demo --quiet


In [8]:
%%bash
# Use the IM2REC script to convert our images into RecordIO files

cd $BASE_DIR

# First we need to create two LST files (training and test lists), noting the correct label class for each image
# We'll also save the output of the LST files command, since it includes a list of all of our label classes
echo "Creating LST files"
python $IM2REC --list --recursive --pass-through --test-ratio=0.3 --train-ratio=0.7 $DATASET_NAME $DATASET_NAME > ${DATASET_NAME}_classes

echo "Label classes:"
cat ${DATASET_NAME}_classes

# Then we create RecordIO files from the LST files
echo "Creating RecordIO files"
python $IM2REC --num-thread=4 ${DATASET_NAME}_train.lst $DATASET_NAME
python $IM2REC --num-thread=4 ${DATASET_NAME}_test.lst $DATASET_NAME
ls -lh *.rec

Creating LST files
Label classes:
bottle 0
card 1
human 2
Creating RecordIO files
Creating .rec file from /tmp/websummit-live-demo_train.lst in /tmp
time: 0.013512134552001953  count: 0
Creating .rec file from /tmp/websummit-live-demo_test.lst in /tmp
time: 0.0011723041534423828  count: 0
-rw-rw-r-- 1 ec2-user ec2-user 380K Jan 30 09:04 websummit-live-demo_test.rec
-rw-rw-r-- 1 ec2-user ec2-user 876K Jan 30 09:04 websummit-live-demo_train.rec


In [9]:
# Upload our train and test RecordIO files to S3 in the bucket that our sagemaker session is using
bucket = sess.default_bucket()
prefix = 'ic-transfer-learning'

s3train_path = 's3://{}/{}/train/'.format(bucket, prefix)
s3validation_path = 's3://{}/{}/validation/'.format(bucket, prefix)

# Clean up any existing data
!aws s3 rm s3://{bucket}/{prefix}/train --recursive
!aws s3 rm s3://{bucket}/{prefix}/validation --recursive

# Upload the rec files to the train and validation channels
!aws s3 cp /tmp/{dataset_name}_train.rec $s3train_path
!aws s3 cp /tmp/{dataset_name}_test.rec $s3validation_path

delete: s3://sagemaker-us-west-2-541003905521/ic-transfer-learning/train/websummit-live-demo_train.rec
delete: s3://sagemaker-us-west-2-541003905521/ic-transfer-learning/validation/websummit-live-demo_test.rec
upload: ../../../tmp/websummit-live-demo_train.rec to s3://sagemaker-us-west-2-541003905521/ic-transfer-learning/train/websummit-live-demo_train.rec
upload: ../../../tmp/websummit-live-demo_test.rec to s3://sagemaker-us-west-2-541003905521/ic-transfer-learning/validation/websummit-live-demo_test.rec


### Configuring the data for our model training to use


In [10]:
train_data = sagemaker.session.s3_input(
    s3train_path, 
    distribution='FullyReplicated', 
    content_type='application/x-recordio', 
    s3_data_type='S3Prefix'
)

validation_data = sagemaker.session.s3_input(
    s3validation_path, 
    distribution='FullyReplicated', 
    content_type='application/x-recordio', 
    s3_data_type='S3Prefix'
)

data_channels = {'train': train_data, 'validation': validation_data}

## Training

In [11]:
s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)
image_classifier = sagemaker.estimator.Estimator(
    training_image,
    role, 
    train_instance_count=1, 
    train_instance_type='ml.p3.2xlarge',
    output_path=s3_output_location,
    sagemaker_session=sess
)

In [12]:
num_classes=! ls -l {base_dir}/{dataset_name} | wc -l
num_classes=int(num_classes[0]) - 1

num_training_samples=! cat {base_dir}/{dataset_name}_train.lst | wc -l
num_training_samples = int(num_training_samples[0])

# Learn more about the Sagemaker built-in Image Classifier hyperparameters here: https://docs.aws.amazon.com/sagemaker/latest/dg/IC-Hyperparameter.html

# These hyperparameters we won't want to change, as they define things like
# the size of the images we'll be sending for input, the number of training classes we have, etc.
base_hyperparameters=dict(
    use_pretrained_model=1,
    image_shape='3,224,224',
    num_classes=num_classes,
    num_training_samples=num_training_samples,
)

# These are hyperparameters we may want to tune, as they can affect the model training success:
hyperparameters={
    **base_hyperparameters, 
    **dict(
        epochs=30,
        learning_rate=0.001,
        mini_batch_size=5,
    )
}


image_classifier.set_hyperparameters(**hyperparameters)

hyperparameters

{'use_pretrained_model': 1,
 'image_shape': '3,224,224',
 'num_classes': 3,
 'num_training_samples': 62,
 'epochs': 30,
 'learning_rate': 0.001,
 'mini_batch_size': 5}

### (Optional) Perform Hyperparameter Tuning

Often, you might not know which values for hyperparameters like `learning_rate` and `mini_batch_size` will yield acceptible results. Traditionally, this meant manually running many training jobs with different hyperparameter values, looking at each trained model's performance, and then picking a winner. 

This type of manual tuning is _very_ time consuming, so you can automate this process using automatic model tuning with SageMaker. Here's some example code to illustrate how to start one of these jobs using the SageMaker Python SDK.

In [25]:
from sagemaker.tuner import HyperparameterTuner, IntegerParameter, CategoricalParameter, ContinuousParameter
hyperparameter_ranges = {'optimizer': CategoricalParameter(['sgd', 'adam']),
                         'learning_rate': ContinuousParameter(0.0001, 0.2),
                         'mini_batch_size': IntegerParameter(2, 30),
                        }

objective_metric_name = 'validation:accuracy'

tuner = HyperparameterTuner(image_classifier,
                            objective_metric_name,
                            hyperparameter_ranges,

                            max_jobs=50,
                            max_parallel_jobs=3)

tuner.fit(inputs=data_channels, logs=True, include_cls_metadata=False)

INFO:sagemaker:Creating hyperparameter tuning job with name: image-classification-190130-0244


### Start the training
This will take some time because it's provisioning a new container runtime to train our model, then the actual training happens, then the trained model gets uploaded to S3 and the container is shut down.

In [13]:
%%time

image_classifier.fit(inputs=data_channels, logs=True)

job = image_classifier.latest_training_job
model_path = f"{base_dir}/{job.name}"

print(f"\n\n Finished training! The model is available for download at: {image_classifier.output_path}/{job.name}/output/model.tar.gz")

INFO:sagemaker:Creating training-job with name: image-classification-2019-01-30-09-05-07-056


2019-01-30 09:05:07 Starting - Starting the training job...
2019-01-30 09:05:08 Starting - Launching requested ML instances......
2019-01-30 09:06:08 Starting - Preparing the instances for training......
2019-01-30 09:07:29 Downloading - Downloading input data...
2019-01-30 09:07:40 Training - Downloading the training image.....
Docker entrypoint called with argument(s): train
2019-01-30 09:08:53 Reading default configuration from /opt/amazon/lib/python2.7/site-packages/image_classification/default-input.json: {u'beta_1': 0.9, u'gamma': 0.9, u'beta_2': 0.999, u'optimizer': u'sgd', u'use_pretrained_model': 0, u'eps': 1e-08, u'epochs': 30, u'lr_scheduler_factor': 0.1, u'num_layers': 152, u'image_shape': u'3,224,224', u'precision_dtype': u'float32', u'mini_batch_size': 32, u'weight_decay': 0.0001, u'learning_rate': 0.1, u'momentum': 0}
[01/30/2019 09:08:53 INFO 139863063271232] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/image_classification/default-input.js

## Deploy the trained model

In [26]:
%%time
# Deploying a model to an endpoint takes a few minutes to complete

deployed_endpoint = image_classifier.deploy(
    initial_instance_count = 1,
    instance_type = 'ml.t2.medium'
)

INFO:sagemaker:Creating model with name: image-classification-2019-01-30-03-48-39-370
INFO:sagemaker:Creating endpoint with name image-classification-2019-01-30-02-14-45-646


---------------------------------------------------------------------------------------------------------------!CPU times: user 538 ms, sys: 27.9 ms, total: 566 ms
Wall time: 9min 21s


### Calling a deployed endpoint from Python code

In [ ]:
import json
import numpy as np
import os

def classify_deployed(file_name, classes):
    payload = None
    with open(file_name, 'rb') as f:
        payload = f.read()
        payload = bytearray(payload)

    deployed_endpoint.content_type = 'application/x-image'
    result = json.loads(deployed_endpoint.predict(payload))
    best_prob_index = np.argmax(result)
    return (classes[best_prob_index], result[best_prob_index])



### Clean up

When we're done with the endpoint, we can just delete it and the backing instances will be released.  Run the following cell to delete the endpoint.

In [27]:
deployed_endpoint.delete_endpoint()

INFO:sagemaker:Deleting endpoint with name: image-classification-2019-01-30-02-14-45-646
